# Transfer Learning

Transfer learning dibuat untuk memecahkan masalah computer vision, model yang telah dilatih pada dataset berukuran besar yang berisi gambar umum mampu dipakai sebagai model dasar yang membantu mengenali fitu/bentuk benda pada dunia nyata

In [ ]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pengembangan_academy/Chessman-image-dataset.zip \
  -O /tmp/Chessman-image-dataset.zip

--2021-06-16 15:13:30--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60684125 (58M) [application/zip]
Saving to: ‘/tmp/Chessman-image-dataset.zip’

/tmp/Chessman-image 100%[===================>]  57.87M  1.61MB/s    in 41s     

2021-06-16 15:14:11 (1.42 MB/s) - ‘/tmp/Chessman-image-dataset.zip’ saved [60684125/60684125]



In [ ]:
import os
import zipfile as zf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

local_zip = '/tmp/Chessman-image-dataset.zip'
zip_ref = zf.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

train_dir = os.path.join('/tmp/Chessman-image-dataset/Chess')
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    zoom_range = 0.2,
    shear_range = 0.2,
    fill_mode = 'nearest',
    validation_split = 0.1
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 8,
    class_mode = 'categorical',
    subset = 'training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 16,
    class_mode = 'categorical',
    subset = 'validation'
)

Found 499 images belonging to 6 classes.
Found 52 images belonging to 6 classes.


Disini kita mulai mengimplementasikan transfer learning. Untuk model yang kita pilih sebagai model dasar transfer learning adalah ResNet152V2. Model ini memiliki 152 layer dan tersedia di library keras.

Untuk mengimplementasikan transfer learning kita perlu menambahkan 2 baris kode berbeda. Layer pertama pada model kita adalah model yang kita pakai untuk transfer learning. Kita cukup memanggil kelas ResNet152V2 dan mengisi parameter:



> weight : parameter weight kita mengisi nilai 'imagenet' yang berarti kita akan menggunakan model ResNet152V2


> include_top : parameter ini bernilai boolean. Maksudnya adalah jika ingin tetap memakai layer terakhir dari model resnet


> input_tensor : sesuai dengan namanya parameter ini menspesifikasikan ukuran dari input







In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2

model = tf.keras.models.Sequential([
                                    ResNet152V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(150, 150, 3))),
                                    tf.keras.layers.Dropout(0.4),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(256, activation='relu'),
                                    tf.keras.layers.Dense(6, activation='softmax')
])

model.layers[0].trainable = False

234553344/234545216 [==============================] - 3s 0us/step


In [ ]:
model.compile(
    optimizer = tf.optimizers.Adam(),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data = validation_generator,
    epochs = 50,
    verbose = 2
)

Epoch 1/50
63/63 - 123s - loss: 6.4987 - accuracy: 0.4469 - val_loss: 1.2659 - val_accuracy: 0.6923
Epoch 2/50
63/63 - 112s - loss: 1.5559 - accuracy: 0.6794 - val_loss: 1.2446 - val_accuracy: 0.6923
Epoch 3/50
63/63 - 112s - loss: 1.3735 - accuracy: 0.6934 - val_loss: 1.1046 - val_accuracy: 0.6154
Epoch 4/50
63/63 - 112s - loss: 0.7160 - accuracy: 0.7956 - val_loss: 0.7700 - val_accuracy: 0.7500
Epoch 5/50
63/63 - 112s - loss: 0.5432 - accuracy: 0.8377 - val_loss: 0.8445 - val_accuracy: 0.7308
Epoch 6/50
63/63 - 112s - loss: 0.5456 - accuracy: 0.8437 - val_loss: 0.7880 - val_accuracy: 0.7115
Epoch 7/50
63/63 - 112s - loss: 0.4080 - accuracy: 0.8617 - val_loss: 0.8260 - val_accuracy: 0.7885
Epoch 8/50
63/63 - 112s - loss: 0.5764 - accuracy: 0.8497 - val_loss: 0.7371 - val_accuracy: 0.7500
Epoch 9/50
63/63 - 112s - loss: 0.5623 - accuracy: 0.8637 - val_loss: 0.6751 - val_accuracy: 0.8462
Epoch 10/50
63/63 - 112s - loss: 0.5967 - accuracy: 0.8577 - val_loss: 1.0616 - val_accuracy: 0.7692

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7fc608f3c9e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Epoch 31/50
